<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#TV-Script-Generation" data-toc-modified-id="TV-Script-Generation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>TV Script Generation</a></span><ul class="toc-item"><li><span><a href="#Get-the-Data" data-toc-modified-id="Get-the-Data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Get the Data</a></span></li><li><span><a href="#Explore-the-Data" data-toc-modified-id="Explore-the-Data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Explore the Data</a></span></li><li><span><a href="#Implement-Preprocessing-Functions" data-toc-modified-id="Implement-Preprocessing-Functions-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Implement Preprocessing Functions</a></span><ul class="toc-item"><li><span><a href="#Lookup-Table" data-toc-modified-id="Lookup-Table-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Lookup Table</a></span></li><li><span><a href="#Tokenize-Punctuation" data-toc-modified-id="Tokenize-Punctuation-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Tokenize Punctuation</a></span></li></ul></li><li><span><a href="#Preprocess-all-the-data-and-save-it" data-toc-modified-id="Preprocess-all-the-data-and-save-it-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Preprocess all the data and save it</a></span></li></ul></li><li><span><a href="#Check-Point" data-toc-modified-id="Check-Point-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Check Point</a></span><ul class="toc-item"><li><span><a href="#Build-the-Neural-Network" data-toc-modified-id="Build-the-Neural-Network-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Build the Neural Network</a></span><ul class="toc-item"><li><span><a href="#Check-the-Version-of-TensorFlow-and-Access-to-GPU" data-toc-modified-id="Check-the-Version-of-TensorFlow-and-Access-to-GPU-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Check the Version of TensorFlow and Access to GPU</a></span></li><li><span><a href="#Batches" data-toc-modified-id="Batches-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Batches</a></span></li></ul></li><li><span><a href="#Neural-Network-Training" data-toc-modified-id="Neural-Network-Training-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Neural Network Training</a></span><ul class="toc-item"><li><span><a href="#Hyperparameters" data-toc-modified-id="Hyperparameters-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Hyperparameters</a></span></li><li><span><a href="#Build-the-Graph" data-toc-modified-id="Build-the-Graph-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Build the Graph</a></span></li></ul></li></ul></li><li><span><a href="#The-TV-Script-is-Nonsensical" data-toc-modified-id="The-TV-Script-is-Nonsensical-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>The TV Script is Nonsensical</a></span></li><li><span><a href="#Submitting-This-Project" data-toc-modified-id="Submitting-This-Project-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Submitting This Project</a></span></li></ul></div>

# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
from time import time

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    conj=set()
    vocab_to_int=dict()
    int_to_vocab=dict()
    for word in text:
        conj.add(word)
        
    for i,word in enumerate(conj):
        vocab_to_int[word]=i
        int_to_vocab[i]=word
        
    return (vocab_to_int, int_to_vocab)



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    pont=dict()
    pont["."]="||period||"
    pont[","]="||comma||"
    pont["\""]="||quotation_mark||"
    pont[";"]="||semicolon||"
    pont["!"]="||exclamation_mark||"
    pont["?"]="||question_mark||"
    pont["("]="||left_parentheses||"
    pont[")"]="||right_parentheses||"
    pont["--"]="||dash||"
    pont["\n"]="||return||"
    return pont

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.3'), 'Please use TensorFlow version 1.3 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 2.0.0
Default GPU Device: /device:GPU:0


In [8]:
class TextGenerator(tf.keras.models.Model):
    def __init__(self, num_layers, rnn_size, batch_size, vocab_size, embed_dim, seq_length):
        super(TextGenerator, self).__init__()
        
        self.batch_size = batch_size
        self.rnn_size = rnn_size
        self.num_layers = num_layers
        
        self.embed = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim, 
                                               input_length=seq_length)
        
        self.rnn_layers = [tf.keras.layers.LSTM(units=rnn_size, batch_size=batch_size, 
                                                return_sequences=True) 
                           for _ in range(num_layers)]
        
        self.fc = tf.keras.layers.Dense(units=vocab_size)
        
        self.act = tf.keras.layers.Softmax()
        
    def call(self, x):
        
        x = self.embed(x)
        for i in range(self.num_layers):
                x = self.rnn_layers[i](x)
        
        logits = self.fc(x)
        output = self.act(logits)
        
        return logits, output

### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [9]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    num=len(int_text)//(batch_size*seq_length)
    text=np.array(int_text[:num*(seq_length*batch_size)])
    inputs=text.reshape(-1,seq_length)
    target=np.roll(text,-1).reshape(-1,seq_length)
    output=np.zeros(shape=(num, 2, batch_size, seq_length),dtype=np.int)
    for i in range(inputs.shape[0]):
        nb=i%num
        y=i//num
        output[nb,0,y,:]=inputs[i,:]
        output[nb,1,y,:]=target[i,:]
    return output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [60]:
num_epochs = 130
batch_size = 128
rnn_size = 128
embed_dim = 128
seq_length = 16
learning_rate = 0.01
show_every_n_batches = 50
sample_every_n_batches = 500
num_layers = 2
vocab_size = len(vocab_to_int)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [62]:
train_model = TextGenerator(num_layers, rnn_size, batch_size, vocab_size, embed_dim, seq_length)
opt = tf.keras.optimizers.Adam(learning_rate)

In [63]:
def loss_func(y_true, logits):
    return tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)(y_true, logits)

In [64]:
def sample_from_model(model, length=20):
    a = np.random.randint(vocab_size)
    sequence = []
    for _ in range(length):
        x = np.array(a).reshape(1,1)
        a = tf.argmax(model(x)[1], 2).numpy()[0][0]
        sequence.append(int_to_vocab[a])
    
    return ' '.join(sequence)

In [67]:
itr = 0
losses = []
start = time()
for e in range(num_epochs):
    batches = get_batches(int_text, batch_size, seq_length)
    for batch in batches:
        itr+=1
        X, y = batch[0], batch[1]
        with tf.GradientTape() as tape:
            logits, outputs = model(X)
            loss = loss_func(y, logits)
            
        losses.append(loss)
        grads = tape.gradient(loss, model.trainable_variables)
        opt.apply_gradients(zip(grads, model.trainable_variables))
        if itr%show_every_n_batches==0:
            print("Epoch: {} ... Loss: {:.2f} ... Time_per_batch: {:.2f}sec/batch".format(e+1, np.mean(losses[-show_every_n_batches:]), (time()-start)/show_every_n_batches))
            start = time()
            
        if itr%sample_every_n_batches==0:
            print(sample_from_model(train_model))
            print('\n\n')

Epoch: 2 ... Loss: 3.35 ... Time_per_batch: 0.11sec/batch
Epoch: 4 ... Loss: 3.23 ... Time_per_batch: 0.12sec/batch
Epoch: 5 ... Loss: 2.95 ... Time_per_batch: 0.12sec/batch
Epoch: 7 ... Loss: 2.88 ... Time_per_batch: 0.12sec/batch
Epoch: 8 ... Loss: 2.71 ... Time_per_batch: 0.11sec/batch
Epoch: 10 ... Loss: 2.61 ... Time_per_batch: 0.12sec/batch
Epoch: 11 ... Loss: 2.46 ... Time_per_batch: 0.12sec/batch
Epoch: 13 ... Loss: 2.38 ... Time_per_batch: 0.11sec/batch
Epoch: 14 ... Loss: 2.30 ... Time_per_batch: 0.11sec/batch
Epoch: 16 ... Loss: 2.22 ... Time_per_batch: 0.11sec/batch
ripping smiles good-looking batmobile cover dollar nein stink spending hook smiles good-looking batmobile cover dollar nein stink spending hook smiles



Epoch: 17 ... Loss: 2.12 ... Time_per_batch: 0.12sec/batch
Epoch: 19 ... Loss: 2.04 ... Time_per_batch: 0.12sec/batch
Epoch: 20 ... Loss: 1.94 ... Time_per_batch: 0.12sec/batch
Epoch: 22 ... Loss: 1.88 ... Time_per_batch: 0.12sec/batch
Epoch: 23 ... Loss: 1.82 

KeyboardInterrupt: 

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.